In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import Tuple, Optional, Union
import glob

In [ ]:
def compute_horizontal_gradient_sum(image: np.ndarray) -> np.ndarray:
    """
    Compute the sum of horizontal gradient for each row of the image.
    
    Args:
        image (np.ndarray): Input image (grayscale)
        
    Returns:
        np.ndarray: Array of gradient sums for each row
    """
    # Compute gradient in y-direction (vertical)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Take absolute value of gradient
    abs_sobel_y = np.abs(sobel_y)
    
    # Sum across each row to get a 1D array of gradient magnitudes
    gradient_sum = np.sum(abs_sobel_y, axis=1)
    
    return gradient_sum


In [ ]:
def find_roof_line(gradient_sum: np.ndarray, window_size: int = 11, 
                   threshold_factor: float = 1.5) -> int:
    """
    Find the roof line by detecting significant changes in gradient sums.
    
    Args:
        gradient_sum (np.ndarray): Array of gradient sums for each row
        window_size (int): Size of window for local maxima detection
        threshold_factor (float): Factor to determine significant gradient changes
        
    Returns:
        int: Row index representing the roof line
    """
    # Smooth the gradient sum to reduce noise
    smoothed = cv2.GaussianBlur(gradient_sum.reshape(-1, 1), (1, window_size), 0).flatten()
    
    # Compute first derivative of the smoothed gradient sum
    derivative = np.diff(smoothed)
    
    # Find local maxima in the derivative (indicating rapid gradient changes)
    # We're only interested in the top half of the image where roof-sky boundaries typically occur
    top_half = len(derivative) // 2
    peaks = []
    
    for i in range(1, top_half - 1):
        if derivative[i] > derivative[i-1] and derivative[i] > derivative[i+1]:
            peaks.append((i, derivative[i]))
    
    # Sort peaks by magnitude (highest gradient change first)
    peaks.sort(key=lambda x: x[1], reverse=True)
    
    # Return the highest significant peak (roof line)
    if peaks:
        return peaks[0][0]
    else:
        # Fallback: return 20% from the top if no significant peak found
        return int(len(gradient_sum) * 0.2)


In [ ]:
def crop_sky_from_image(image: np.ndarray, offset: int = 20) -> Tuple[np.ndarray, int]:
    """
    Crop the sky from a building facade image.
    
    Args:
        image (np.ndarray): Input image (BGR or RGB)
        offset (int): Additional offset above the detected roof line
        
    Returns:
        Tuple[np.ndarray, int]: Cropped image and the crop line position
    """
    # Convert to grayscale if image is color
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Compute gradient sum for each row
    gradient_sum = compute_horizontal_gradient_sum(gray)
    
    # Find roof line
    roof_line = find_roof_line(gradient_sum)
    
    # Apply offset (make sure we don't go out of bounds)
    crop_line = max(0, roof_line - offset)
    
    # Crop the image
    cropped_image = image[crop_line:, :]
    
    return cropped_image, crop_line



In [ ]:
def visualize_gradient_and_crop(image: np.ndarray, crop_line: int) -> np.ndarray:
    """
    Create a visualization of the gradient analysis and cropping result.
    
    Args:
        image (np.ndarray): Original image
        crop_line (int): The detected crop line
        
    Returns:
        np.ndarray: Visualization image
    """
    # Convert to grayscale if image is color
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
        image = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    
    # Compute gradient sum
    gradient_sum = compute_horizontal_gradient_sum(gray)
    
    # Normalize for visualization
    normalized = gradient_sum / np.max(gradient_sum) * image.shape[1] * 0.3
    
    # Create a copy of the image for visualization
    vis_image = image.copy()
    
    # Draw gradient profile
    for i, mag in enumerate(normalized):
        cv2.line(vis_image, (0, i), (int(mag), i), (0, 255, 0), 1)
    
    # Draw crop line
    cv2.line(vis_image, (0, crop_line), (vis_image.shape[1], crop_line), (0, 0, 255), 2)
    
    return vis_image


In [ ]:
def process_and_display_image(image_path: str, offset: int = 20, 
                             show_visualization: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    """
    Process an image file, crop the sky, and optionally display results.
    
    Args:
        image_path (str): Path to the image file
        offset (int): Additional offset above the detected roof line
        show_visualization (bool): Whether to display visualization
        
    Returns:
        Tuple[np.ndarray, np.ndarray]: Original and cropped images
    """
    # Load image
    original = cv2.imread(image_path)
    if original is None:
        raise ValueError(f"Could not load image from {image_path}")
    
    # Crop sky
    cropped, crop_line = crop_sky_from_image(original, offset)
    
    if show_visualization:
        # Create visualization
        vis_image = visualize_gradient_and_crop(original, crop_line)
        
        # Display results
        plt.figure(figsize=(15, 10))
        
        plt.subplot(1, 3, 1)
        plt.title("Original Image")
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.title("Gradient Analysis")
        plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.title("Cropped Image")
        plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return original, cropped


In [ ]:
def batch_process_images(image_paths: list, output_dir: str, offset: int = 20) -> None:
    """
    Process multiple images and save the cropped results.
    
    Args:
        image_paths (list): List of paths to image files
        output_dir (str): Directory to save cropped images
        offset (int): Additional offset above the detected roof line
    """
    import os
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    for image_path in image_paths:
        try:
            # Load image
            original = cv2.imread(image_path)
            if original is None:
                print(f"Could not load image from {image_path}")
                continue
            
            # Crop sky
            cropped, _ = crop_sky_from_image(original, offset)
            
            # Save cropped image
            filename = os.path.basename(image_path)
            output_path = os.path.join(output_dir, f"cropped_{filename}")
            cv2.imwrite(output_path, cropped)
            
            print(f"Processed {filename} -> {output_path}")
            
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

In [ ]:
image_paths = glob.glob("./cropped_images/*.jpg")
output_dir = "./cropped_images/cropped_sky"
batch_process_images(image_paths, offset=30, output_dir=output_dir)

Processed pano_000117_000042_building_cropped.jpg -> ./cropped_images/cropped_sky\cropped_pano_000117_000042_building_cropped.jpg
Processed pano_000117_000043_building_cropped.jpg -> ./cropped_images/cropped_sky\cropped_pano_000117_000043_building_cropped.jpg
Processed pano_000117_000045_building_cropped.jpg -> ./cropped_images/cropped_sky\cropped_pano_000117_000045_building_cropped.jpg
